In [1]:
! pip install textblob
! pip install pandas
! pip install nltk
! pip install sklearn
! pip install contractions
! pip install emoji 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=606e3b7400f3c7800bf94cc972e7029d96d4f48958e6a65b83a0272da11731ab
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 4.1 MB/s 
     |████████████████████████████████| 106 kB 52.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [2]:
# import required libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import re
import contractions
import emoji
import string 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('sentiwordnet')
from nltk.corpus import stopwords, wordnet
from nltk.corpus import sentiwordnet as swn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


In [4]:
raw_data = pd.read_csv('train_df_imbalanced.csv')
raw_data.head()

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity
0,9406,B08NLXR9V5,4,Girl Alone,One person found this helpful,"My first Blake Pierce book was enjoyable, fast...",True,mystery,Language.ENGLISH,Girl Alone. My first Blake Pierce book was enj...,1
1,5701,670062510,5,A great book for young people. It has a great...,NaN,My favorite book when I was young. I read it ...,True,children,Language.ENGLISH,A great book for young people. It has a great...,1
2,13621,1542046599,1,More gripes than hypes,One person found this helpful,This is the first book I‚Äôve read by this aut...,True,mystery,Language.ENGLISH,More gripes than hypes. This is the first book...,-1
3,5021,399226907,5,Add this book to your collection,NaN,Cute and educational book to teach counting an...,True,children,Language.ENGLISH,Add this book to your collection. Cute and edu...,1
4,21354,125030170X,2,Just okay.,NaN,This is one of those books you can read in a c...,False,children,Language.ENGLISH,Just okay.. This is one of those books you can...,0


In [5]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF" 
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def remove_stopwords(reviews):
    STOPWORDS = stopwords.words('english')
    STOPWORDS.remove('not')
    STOPWORDS.remove('is')
    STOPWORDS.remove('but')
    if STOPWORDS is None:
        STOPWORDS = set(stopwords.words('english'))
    return ' '.join([word for word in reviews.split() if word not in STOPWORDS])
def remove_extra_whitespace(reviews):
    return " ".join(reviews.split())

def get_wordnet_pos(text):
    # Map POS tag to first character lemmatize() accepts
    tags = nltk.pos_tag(text)
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    tags = [tag_dict.get(tag[1][0],  wordnet.NOUN) for tag in tags]
    return tags

def lemmaSentence(reviews):
    lemmatizer = WordNetLemmatizer()
    lemma_text = ''
    tok_text = word_tokenize(reviews)
    tags = get_wordnet_pos(tok_text)
    for i in range(len(tok_text)):
        lemma_text = lemma_text + ' ' + lemmatizer.lemmatize(tok_text[i], tags[i])
    return lemma_text[1:]

def lower_case(review):
    
    return review.lower()

# change contraction words such sa I'm = I am, shouldn't = should not
def change_contractions(review):
    
    expanded_words = [contractions.fix(word) for word in review.split()]

    expanded_review = ' '.join(expanded_words)
    return expanded_review

# Remove Punctuations
def remove_punctuations(review):
    
    new_review = review.translate(str.maketrans('', '', string.punctuation))
    return new_review

# Remove numbers
def remove_numbers(review):
    
    mapping = str.maketrans('', '', string.digits)
    new_review = review.translate(mapping)
    
    return new_review

def change_to_apostrophe(review):

    funny_symbol = '‚äô'
    return re.sub(funny_symbol, "'", review)

  

In [6]:
# pre-processing using code from Train_Test_Processing.ipynb
def clean_text(data, textcol):

    data['cleaned_text'] = data[textcol].apply(lower_case)
    data['cleaned_text'] = data[textcol].apply(change_contractions)
    data['cleaned_text'] = data[textcol].apply(remove_emojis)
    data['cleaned_text'] = data[textcol].apply(remove_punctuations)
    data['cleaned_text'] = data[textcol].apply(remove_numbers)
    data['cleaned_text'] = data[textcol].apply(remove_stopwords)
    data['cleaned_text'] = data[textcol].apply(remove_extra_whitespace)
    data['cleaned_text'] = data[textcol].apply(lemmaSentence)
    data['cleaned_text'] = data[textcol].apply(change_to_apostrophe)
    
    return data

In [7]:
clean_data = clean_text(raw_data, 'concat_review')
clean_data.head()

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text
0,9406,B08NLXR9V5,4,Girl Alone,One person found this helpful,"My first Blake Pierce book was enjoyable, fast...",True,mystery,Language.ENGLISH,Girl Alone. My first Blake Pierce book was enj...,1,Girl Alone. My first Blake Pierce book was enj...
1,5701,670062510,5,A great book for young people. It has a great...,NaN,My favorite book when I was young. I read it ...,True,children,Language.ENGLISH,A great book for young people. It has a great...,1,A great book for young people. It has a great...
2,13621,1542046599,1,More gripes than hypes,One person found this helpful,This is the first book I‚Äôve read by this aut...,True,mystery,Language.ENGLISH,More gripes than hypes. This is the first book...,-1,More gripes than hypes. This is the first book...
3,5021,399226907,5,Add this book to your collection,NaN,Cute and educational book to teach counting an...,True,children,Language.ENGLISH,Add this book to your collection. Cute and edu...,1,Add this book to your collection. Cute and edu...
4,21354,125030170X,2,Just okay.,NaN,This is one of those books you can read in a c...,False,children,Language.ENGLISH,Just okay.. This is one of those books you can...,0,Just okay.. This is one of those books you can...


In [8]:
def add_subjectivity(row):
  review = TextBlob(row['cleaned_text'])
  subjectivity = review.sentiment.subjectivity
  # if subjectivity < 0.5:
  #   val = 0 # objective, non-opinionated
  # else:
  #   val = 1 # subjective, opinionated
  # return val
  return subjectivity 

In [9]:
def add_polarity(row):
  review = TextBlob(row['cleaned_text'])
  polarity = review.sentiment.polarity
  # if polarity < 0.0:
  #   val = 0 # negative
  # else:
  #   val = 1 # positive
  # return val
  return polarity

In [10]:
clean_data['tb_subjectivity'] = clean_data.apply(add_subjectivity, axis=1)
clean_data['tb_polarity'] = clean_data.apply(add_polarity, axis=1)
clean_data.head()

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text,tb_subjectivity,tb_polarity
0,9406,B08NLXR9V5,4,Girl Alone,One person found this helpful,"My first Blake Pierce book was enjoyable, fast...",True,mystery,Language.ENGLISH,Girl Alone. My first Blake Pierce book was enj...,1,Girl Alone. My first Blake Pierce book was enj...,0.458333,0.060714
1,5701,670062510,5,A great book for young people. It has a great...,NaN,My favorite book when I was young. I read it ...,True,children,Language.ENGLISH,A great book for young people. It has a great...,1,A great book for young people. It has a great...,0.539394,0.271633
2,13621,1542046599,1,More gripes than hypes,One person found this helpful,This is the first book I‚Äôve read by this aut...,True,mystery,Language.ENGLISH,More gripes than hypes. This is the first book...,-1,More gripes than hypes. This is the first book...,0.367403,0.088295
3,5021,399226907,5,Add this book to your collection,NaN,Cute and educational book to teach counting an...,True,children,Language.ENGLISH,Add this book to your collection. Cute and edu...,1,Add this book to your collection. Cute and edu...,0.580000,0.355000
4,21354,125030170X,2,Just okay.,NaN,This is one of those books you can read in a c...,False,children,Language.ENGLISH,Just okay.. This is one of those books you can...,0,Just okay.. This is one of those books you can...,0.500000,0.500000


In [11]:
objective_reviews = clean_data.loc[(clean_data['tb_subjectivity'] < 0.3)  & (clean_data['tb_subjectivity'] > 0.0)]

In [12]:
objective_reviews

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text,tb_subjectivity,tb_polarity
11,17316,1501128019,5,just Read IT Already,NaN,I‚Äôve passed by this book time and again‚Ä¶di...,True,romance,Language.ENGLISH,just Read IT Already. I‚Äôve passed by this bo...,1,just Read IT Already. I‚Äôve passed by this bo...,0.244444,0.122222
19,5364,399226907,5,Thumbs up,NaN,My 2 year old loves this book.,True,children,Language.ENGLISH,Thumbs up. My 2 year old loves this book.,1,Thumbs up. My 2 year old loves this book.,0.200000,0.100000
106,11637,1542046599,4,Four Stars,NaN,Started out slow but after the first few chapt...,True,mystery,Language.ENGLISH,Four Stars. Started out slow but after the fir...,1,Four Stars. Started out slow but after the fir...,0.277778,-0.100000
143,24219,425284700,1,That the problem was taken care of,NaN,I couldn't get past the Prelude to continue to...,True,children,Language.ENGLISH,That the problem was taken care of. I couldn't...,-1,That the problem was taken care of. I couldn't...,0.291667,0.000000
157,2182,1451673310,5,I wish Amazon could sell translated to othe...,NaN,I read all Bradbury stories tanslated to Rus...,True,humor_entertainment,Language.ENGLISH,I wish Amazon could sell translated to othe...,-1,I wish Amazon could sell translated to othe...,0.239583,0.020833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20600,5727,670062510,5,Classic book,NaN,Came as expected. Gift for my daughter.,True,children,Language.ENGLISH,Classic book. Came as expected. Gift for my da...,1,Classic book. Came as expected. Gift for my da...,0.283333,0.033333
20653,9612,B08NLXR9V5,5,Highly recommend,NaN,Very well done the suspense was me not wanting...,True,mystery,Language.ENGLISH,Highly recommend. Very well done the suspense ...,1,Highly recommend. Very well done the suspense ...,0.282222,0.051111
20891,7226,B01M0JHBEG,5,heartwarming book,NaN,My grandson who is three years old loves these...,True,children,Language.ENGLISH,heartwarming book. My grandson who is three ye...,1,heartwarming book. My grandson who is three ye...,0.200000,0.100000
20931,192,451524934,5,Current Events,NaN,I read when younger like it was fantasy...now ...,True,humor_entertainment,Language.ENGLISH,Current Events. I read when younger like it wa...,0,Current Events. I read when younger like it wa...,0.200000,0.000000


In [157]:
# objective_reviews.to_csv('objective_reviews.csv')

In [13]:
objective_reviews['ratingScore'].value_counts()

5    241
1    235
4     68
2     38
3     33
Name: ratingScore, dtype: int64

In [14]:
i = 0
print(objective_reviews.iloc[i]['concat_review'])
print('subjectivity: ', objective_reviews.iloc[i]['tb_subjectivity'])
print('polarity: ', objective_reviews.iloc[i]['tb_polarity'])
print('stars: ', objective_reviews.iloc[i]['ratingScore'])

just Read IT Already. I‚Äôve passed by this book time and again‚Ä¶didn‚Äôt want the aggravation of going back and forth between youth and adult  But last night I sampled it and finished all but the last chapter as light peaked under the window shade. I had to save the last scoop of dessert ofElliott and Macy. The pace increased with each  encounter and the alternative times built tension. The love felt real and the loss crushed and the forgiveness saved.  it
subjectivity:  0.2444444444444444
polarity:  0.1222222222222222
stars:  5


In [15]:
i = 0
review = TextBlob(objective_reviews.iloc[i]['concat_review'])
for w in review.sentences:
  print(w)
  print(w.sentiment)

just Read IT Already.
Sentiment(polarity=0.0, subjectivity=0.0)
I‚Äôve passed by this book time and again‚Ä¶didn‚Äôt want the aggravation of going back and forth between youth and adult  But last night I sampled it and finished all but the last chapter as light peaked under the window shade.
Sentiment(polarity=0.1, subjectivity=0.22666666666666666)
I had to save the last scoop of dessert ofElliott and Macy.
Sentiment(polarity=0.0, subjectivity=0.06666666666666667)
The pace increased with each  encounter and the alternative times built tension.
Sentiment(polarity=0.0, subjectivity=0.0)
The love felt real and the loss crushed and the forgiveness saved.
Sentiment(polarity=0.19999999999999998, subjectivity=0.3333333333333333)
it
Sentiment(polarity=0.0, subjectivity=0.0)


In [16]:
ambiguous = clean_data.loc[clean_data['tb_subjectivity'] == 0]

In [17]:
len(ambiguous)

366

In [19]:
ambiguous['ratingScore'].value_counts()

1    178
5    127
4     35
3     14
2     12
Name: ratingScore, dtype: int64

In [20]:
i = 0
print(ambiguous.iloc[i]['concat_review'])
print(ambiguous.iloc[i]['tb_polarity'])

Must read. Must read
0.0


In [25]:
pos=neg=obj=count=0

postagging = []

for review in clean_data['cleaned_text']:
  l = word_tokenize(review)
  postagging.append(nltk.pos_tag(l))

clean_data['pos_tags'] = postagging
clean_data.head()

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text,tb_subjectivity,tb_polarity,pos_tags
0,9406,B08NLXR9V5,4,Girl Alone,One person found this helpful,"My first Blake Pierce book was enjoyable, fast...",True,mystery,Language.ENGLISH,Girl Alone. My first Blake Pierce book was enj...,1,Girl Alone. My first Blake Pierce book was enj...,0.458333,0.060714,"[(Girl, NNP), (Alone, NNP), (., .), (My, PRP$)..."
1,5701,670062510,5,A great book for young people. It has a great...,NaN,My favorite book when I was young. I read it ...,True,children,Language.ENGLISH,A great book for young people. It has a great...,1,A great book for young people. It has a great...,0.539394,0.271633,"[(A, DT), (great, JJ), (book, NN), (for, IN), ..."
2,13621,1542046599,1,More gripes than hypes,One person found this helpful,This is the first book I‚Äôve read by this aut...,True,mystery,Language.ENGLISH,More gripes than hypes. This is the first book...,-1,More gripes than hypes. This is the first book...,0.367403,0.088295,"[(More, RBR), (gripes, NNS), (than, IN), (hype..."
3,5021,399226907,5,Add this book to your collection,NaN,Cute and educational book to teach counting an...,True,children,Language.ENGLISH,Add this book to your collection. Cute and edu...,1,Add this book to your collection. Cute and edu...,0.580000,0.355000,"[(Add, VB), (this, DT), (book, NN), (to, TO), ..."
4,21354,125030170X,2,Just okay.,NaN,This is one of those books you can read in a c...,False,children,Language.ENGLISH,Just okay.. This is one of those books you can...,0,Just okay.. This is one of those books you can...,0.500000,0.500000,"[(Just, RB), (okay, RB), (.., VB), (This, DT),..."


In [26]:
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB
  return None 

In [27]:
lemmatizer = WordNetLemmatizer()
# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word, tag):
  wn_tag = penn_to_wn(tag)
  if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
    return []
  lemma = lemmatizer.lemmatize(word, pos=wn_tag)
  if not lemma:
    return []
  #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
  #Synset instances are the groupings of synonymous words that express the same concept. 
  #Some of the words have only one Synset and some have several.
  synsets = wn.synsets(word, pos=wn_tag)
  if not synsets:
    return []
  # Take the first sense, the most common
  synset = synsets[0]
  swn_synset = swn.senti_synset(synset.name())
  return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [28]:
senti_score = []

for pos_val in clean_data['pos_tags']:
  senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
  for score in senti_val:
    try:
      pos = pos + score[1] #positive score is stored at 2nd position
      neg = neg + score[2] #negative score is stored at 3rd position
    except:
      continue
  senti_score.append(pos-neg)
  pos=neg=0

clean_data['senti_score'] = senti_score

In [34]:
overall = []
for i in range(len(clean_data)):
  if (clean_data['senti_score'][i] < 0.1) & (clean_data['senti_score'][i] > -0.1):
    overall.append(0)
  else:
    overall.append(1)
clean_data['swn_subjectivity'] = overall

In [35]:
objective_reviews1 = clean_data.loc[(clean_data['tb_subjectivity'] < 0.3) & (clean_data['swn_subjectivity'] == 0)]

In [36]:
objective_reviews1

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text,tb_subjectivity,tb_polarity,pos_tags,senti_score,swn_subjectivity
339,525,451526341,4,Please enter your headline.,NaN,Please enter your headline.,True,humor_entertainment,Language.ENGLISH,Please enter your headline.. Please enter your...,1,Please enter your headline.. Please enter your...,0.000000,0.0,"[(Please, NNP), (enter, VB), (your, PRP$), (he...",0.0,0
423,6962,1641526270,5,Recommended for preschooler or stay at home to...,NaN,I like the way this book startsAnd my son love...,True,children,Language.ENGLISH,Recommended for preschooler or stay at home to...,1,Recommended for preschooler or stay at home to...,0.000000,0.0,"[(Recommended, VBN), (for, IN), (preschooler, ...",0.0,0
459,294,451524934,5,This book should be read by every American.,NaN,This book reveals the madness occurring in Ame...,True,humor_entertainment,Language.ENGLISH,This book should be read by every American.. T...,-1,This book should be read by every American.. T...,0.000000,0.0,"[(This, DT), (book, NN), (should, MD), (be, VB...",0.0,0
563,20889,1407195573,1,Nope,1,Nope don't do it to yourself or the kid,True,children,Language.ENGLISH,Nope. Nope don't do it to yourself or the kid,-1,Nope. Nope don't do it to yourself or the kid,0.000000,0.0,"[(Nope, NNP), (., .), (Nope, NNP), (do, VBP), ...",0.0,0
568,23692,399562494,1,Holler?,NaN,"""Holler?"" I can't read past the first chapter.",False,children,Language.ENGLISH,"Holler?. ""Holler?"" I can't read past the first...",-1,"Holler?. ""Holler?"" I can't read past the first...",0.291667,0.0,"[(Holler, NNP), (?, .), (., .), (``, ``), (Hol...",0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20513,9673,B08NLXR9V5,4,I use it every time.,NaN,"Reading, I use it every time.",True,mystery,Language.ENGLISH,"I use it every time.. Reading, I use it every ...",1,"I use it every time.. Reading, I use it every ...",0.000000,0.0,"[(I, PRP), (use, VBP), (it, PRP), (every, DT),...",0.0,0
20601,589,451526341,5,Barn,NaN,I play the pig,True,humor_entertainment,Language.ENGLISH,Barn. I play the pig,-1,Barn. I play the pig,0.000000,0.0,"[(Barn, NNP), (., .), (I, PRP), (play, VBP), (...",0.0,0
20843,10241,1542046599,4,Four Stars,NaN,Keeps your attention,True,mystery,Language.ENGLISH,Four Stars. Keeps your attention,1,Four Stars. Keeps your attention,0.000000,0.0,"[(Four, CD), (Stars, NNP), (., .), (Keeps, NNP...",0.0,0
20852,8826,887431453,5,Workbook,NaN,My granddaughter loves it!! She will start kin...,True,children,Language.ENGLISH,Workbook. My granddaughter loves it!! She will...,1,Workbook. My granddaughter loves it!! She will...,0.000000,0.0,"[(Workbook, NNP), (., .), (My, PRP$), (grandda...",0.0,0


In [37]:
objective_reviews1['ratingScore'].value_counts()

1    90
5    63
4    27
2     9
3     5
Name: ratingScore, dtype: int64

In [38]:
i = 0
print(objective_reviews1.iloc[i]['concat_review'])
print('textblob:', objective_reviews1.iloc[i]['tb_subjectivity'])
print('senti score:', objective_reviews1.iloc[i]['senti_score'])
print('stars:', objective_reviews1.iloc[i]['ratingScore'])

Please enter your headline.. Please enter your headline.
textblob: 0.0
senti score: 0.0
stars: 4


In [39]:
testing = clean_data.loc[clean_data['Unnamed: 0'] == 8866]
testing

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity,cleaned_text,tb_subjectivity,tb_polarity,pos_tags,senti_score,swn_subjectivity
18707,8866,887431453,5,:*¬®¬®*:¬∑.EXCELLENT.¬∑:*¬®¬®*:¬∑.PRODUCT.¬∑:*...,NaN,:*¬®¬®*:¬∑.EXCELLENT.¬∑:*¬®¬®*:¬∑.PRODUCT.¬∑:*...,True,children,Language.ENGLISH,:*¬®¬®*:¬∑.EXCELLENT.¬∑:*¬®¬®*:¬∑.PRODUCT.¬∑:*...,1,:*¬®¬®*:¬∑.EXCELLENT.¬∑:*¬®¬®*:¬∑.PRODUCT.¬∑:*...,0.0,0.0,"[(:, :), (*, NN), (¬®¬®, VBZ), (*, NNS), (:, :...",0.0,0


In [40]:
objective_reviews1.to_csv('objective_reviews1.csv')